<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Energy-Consumption-Forecasting/blob/main/Forecasting_Daily_Energy_Demand_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import kagglehub
from kagglehub import KaggleDatasetAdapter



In [ ]:
# Load dataset from Kaggle
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "jeanmidev/smart-meters-in-london",
    "daily_dataset.csv",
)
df.head()

In [ ]:
# Drop LCLid column if it exists
df.drop(columns=['LCLid'], errors='ignore', inplace=True)

In [ ]:
df.head()

In [ ]:
# Convert 'day' column to datetime and set index
df['day'] = pd.to_datetime(df['day'])
df.set_index('day', inplace=True)

# Selecting energy_sum as target variable
df_daily = df[['energy_sum']]


# Normalize data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_daily)

# Prepare dataset for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 30  # Using past 30 days to predict the next day
X, y = create_sequences(df_scaled, seq_length)

# Split data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(X.shape[2])
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=16)


In [ ]:
# Plot training history
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform predictions
y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

# Plot actual vs predicted
plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual Consumption (energy_sum)')
plt.plot(y_pred_inv, label='Predicted Consumption (energy_sum)')
plt.legend()
plt.show()

In [ ]:
from google.colab import files
files.download('energy_forecast_model.h5')